In [1]:
# Colab cell
try:
    import google.colab
    IN_COLAB = True
    !wget -q https://raw.githubusercontent.com/Competencia-de-Climate-Change/WeatherBench/master/src/ml_training.py
    !pip install -q -U zarr
    !pip install -q -U fsspec
    !pip install -q -U gcsfs
    !pip install -q -U xarray
except:
    IN_COLAB = False

In [2]:
import xarray as xr
import fsspec

ds = xr.open_zarr(fsspec.get_mapper('gcs://weather-bench/region_zarr/'), consolidated=True)

print("Tamanño en GB:", ds.nbytes / 1e9)

Tamanño en GB: 19.324476644


In [3]:
ds

<xarray.Dataset>
Dimensions:    (lat: 12, level: 13, lon: 14, time: 350640)
Coordinates:
  * lat        (lat) float64 -70.31 -64.69 -59.06 ... -19.69 -14.06 -8.438
  * level      (level) int32 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * lon        (lon) float64 247.5 253.1 258.8 264.4 ... 303.8 309.4 315.0 320.6
  * time       (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
Data variables:
    lat2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lon2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lsm        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    orography  (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    pv         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    q          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    r          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    slt        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    t          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    t2m        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    tcc        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    u10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    v10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    vo         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    z          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-03-08 20:51:24 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

# Linear Regression for $Z500$, $T850$ and $T2M$

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import pickle

from src.ml_training import *

# from dask_ml.wrappers import ParallelPostFit

def to_pickle(obj, fn):
    with open(fn, 'wb') as f:
        pickle.dump(obj, f)
def read_pickle(fn):
    with open(fn, 'rb') as f:
        return pickle.load(f)
    
results = 'results'
verbose = False

import os
if not os.path.exists(results):
    os.makedirs(results)

/opt/tljh/user/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


## Data Loading

In [5]:
z_500 = ds.z.sel(level=500).drop('level')
t_850 = ds.t.sel(level=850).drop('level')
t2m   = ds.t2m

data = xr.merge([z_500, t_850, t2m])

data_train = data.sel(time=slice('1979', '2016'))
data_test  = data.sel(time=slice('2017', '2018'))

z_test   = load_test_data('z', ds=ds)
t_test   = load_test_data('t', ds=ds)
t2m_test = load_test_data('t2m', ds=ds)

test_data = xr.merge([z_test, t_test, t2m_test])

_, nlat, nlon = data_train.z.shape

nlat, nlon

Selecting from ds...
Selecting from ds...


(12, 14)

In [6]:
# Compute normalization statistics
# Let's only take a sample of the time to speed this up.
data_mean = data_train.isel(time=slice(0, None, 10000)).mean().load()
data_std  = data_train.isel(time=slice(0, None, 10000)).std().load()


# Normalize datasets
data_train = (data_train - data_mean) / data_std
data_test  = (data_test - data_mean)  / data_std

### Define combinations of training data

In [7]:
# Inputs and Outputs
experiments = [
    [['z'], ['z']],
    [['t'], ['t']],
    [['z', 't'], ['z', 't']],
    [['t2m'], ['t2m']],
    [['z', 't', 't2m'], ['z']],
    [['z', 't', 't2m'], ['t']],
    [['z', 't', 't2m'], ['t2m']],
]

lead_time = 3*24 # Predict 3 days from current time
lead_time_str = '3d'

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import MultiTaskLasso, MultiTaskLassoCV
from sklearn.linear_model import MultiTaskElasticNet, MultiTaskElasticNetCV

## Run Experiments

In [9]:
preds  = {
    'lr'        : [],
    'ridge'     : [],
    'ridge_cv'  : [],
    'lasso'     : [],
    'elasticnet': []
}

models = {
    'lr'        : [],
    'ridge'     : [],
    'ridge_cv'  : [],
    'lasso'     : [],
    'elasticnet': []
}

train_mse = {
    'lr'        : [],
    'ridge'     : [],
    'ridge_cv'  : [],
    'lasso'     : [],
    'elasticnet': []
}

test_mse = {
    'lr'        : [],
    'ridge'     : [],
    'ridge_cv'  : [],
    'lasso'     : [],
    'elasticnet': []
}

weighted_mse = {
    'lr'        : [],
    'ridge'     : [],
    'ridge_cv'  : [],
    'lasso'     : [],
    'elasticnet': []
}

model_args = [
    [LinearRegression,    'lr', {}], 
    [Ridge,               'ridge', {}], 
    [RidgeCV,             'ridge_cv', {'alphas': [0.001, 0.01, 1., 10., 50.]}],
    [MultiTaskLasso,      'lasso', {'tol' : 0.001, 'selection' : 'random'}],
    [MultiTaskElasticNet, 'elasticnet', {'tol' : 0.001, 'selection' : 'random'}]
]

for model in model_args:
    model_fun, model_str, extra_args = model
    
    print('Running:', model_str)
    for iter, (i, o) in enumerate(experiments):
        print(f'{iter}: Input variables = {i}; output variables = {o}')
        
        predictions, model_res, mse_train, mse_test = train_regression(model=model_fun,
                                                                       data=[data_train, data_test, nlat, nlon, data_std, data_mean],
                                                                       lead_time_h=lead_time, 
                                                                       input_vars=i, 
                                                                       output_vars=o,
                                                                       data_subsample=5,
                                                                       extra_args=extra_args,
                                                                       verbose=verbose)

        preds[model_str].append(predictions)
        models[model_str].append(model_res)

        # Compute Weighted RMSE: RMSE with a weight that takes into account grid location
        r = compute_weighted_rmse(predictions, test_data).compute()
        if verbose:
            print('; '.join([f'{v} = {r[v].values}' for v in r]) + '\n')

        weighted_mse[model_str].append(r)

        # Save models
        preds_nc_str  = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.nc'
        model_res_str = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.pkl'
        
        predictions.to_netcdf(preds_nc_str)
        to_pickle(model_res, model_res_str)
    print('----------------------------------------------------------------------------')

Running: lr
0: Input variables = ['z']; output variables = ['z']
1: Input variables = ['t']; output variables = ['t']
2: Input variables = ['z', 't']; output variables = ['z', 't']
3: Input variables = ['t2m']; output variables = ['t2m']
4: Input variables = ['z', 't', 't2m']; output variables = ['z']
5: Input variables = ['z', 't', 't2m']; output variables = ['t']
6: Input variables = ['z', 't', 't2m']; output variables = ['t2m']
----------------------------------------------------------------------------
Running: ridge
0: Input variables = ['z']; output variables = ['z']
1: Input variables = ['t']; output variables = ['t']
2: Input variables = ['z', 't']; output variables = ['z', 't']
3: Input variables = ['t2m']; output variables = ['t2m']
4: Input variables = ['z', 't', 't2m']; output variables = ['z']
5: Input variables = ['z', 't', 't2m']; output variables = ['t']
6: Input variables = ['z', 't', 't2m']; output variables = ['t2m']
--------------------------------------------------

In [120]:
for model, ds_list in weighted_mse.items():
    print("Model:", model)
    
    for ds in ds_list:
        print('  Experiment:')
        for var in list(ds.variables):
            print(f'{var:>10} = {round(float(ds[var].values), 4):>5}')
        print()

Model: lr
  Experiment:
         z = 791.4518

  Experiment:
         t = 3.3004

  Experiment:
         z = 787.7357
         t = 3.1999

  Experiment:
       t2m = 2.0013

  Experiment:
         z = 789.3704

  Experiment:
         t = 3.1997

  Experiment:
       t2m = 1.9337

Model: ridge
  Experiment:
         z = 791.0974

  Experiment:
         t = 3.3003

  Experiment:
         z = 787.3135
         t = 3.1988

  Experiment:
       t2m = 2.0011

  Experiment:
         z = 788.9203

  Experiment:
         t = 3.1986

  Experiment:
       t2m = 1.9328

Model: ridge_cv
  Experiment:
         z = 791.0974

  Experiment:
         t = 3.2991

  Experiment:
         z = 787.3137
         t = 3.1988

  Experiment:
       t2m =   2.0

  Experiment:
         z = 788.9205

  Experiment:
         t = 3.1945

  Experiment:
       t2m = 1.9302

Model: lasso
  Experiment:
         z = 962.8299

  Experiment:
         t = 4.0017

  Experiment:
         z = 945.5678
         t = 3.9474

  Exper

In [10]:
# def apply_polinomial(X, deg=3):
#     n = X.shape[0]
#     Phi = np.ones(n)
#     for i in range(1, deg+1):
#         Phi = np.vstack([Phi, X**i])
#     return Phi.T

# def apply_coseno(X, deg=1, T=2):
#     """
#     X: 1d observations
#     d: degree
#     T: period
#     """
#     phi_x = np.ones((len(x), deg+1)) 
#     for d in range(1, deg + 1):
#         phi_x[:, d] = np.cos(d * 2 * np.pi / deg / T * x)
#     return phi_x

# for count, degree in enumerate([3, 4, 5]):
#     model = make_pipeline(PolynomialFeatures(degree), Ridge())
#     model.fit(X, y)
#     y_plot = model.predict(X_plot)
#     plt.plot(x_plot, y_plot, color=colors[count], linewidth=lw,
#              label="degree %d" % degree)

In [59]:
import pandas as pd

results_df = pd.DataFrame(columns=['model', 'input', 'output',  'train_mse', 'test_mse', 'weighted_mse'])

results_df['input']  = list(np.array(experiments, dtype='object')[:, 0])
results_df['output'] = list(np.array(experiments, dtype='object')[:, 1])

In [60]:
results_df

,model,input,output,train_mse,test_mse,weighted_mse
0,NaN,[z],[z],NaN,NaN,NaN
1,NaN,[t],[t],NaN,NaN,NaN
2,NaN,"[z, t]","[z, t]",NaN,NaN,NaN
3,NaN,[t2m],[t2m],NaN,NaN,NaN
4,NaN,"[z, t, t2m]",[z],NaN,NaN,NaN
5,NaN,"[z, t, t2m]",[t],NaN,NaN,NaN
6,NaN,"[z, t, t2m]",[t2m],NaN,NaN,NaN


## Iterative Linear Regression

In [11]:
# def create_iterative_lr(state, model, lead_time_h=6, max_lead_time_h=5*24):
    
#     max_fc_steps = max_lead_time_h // lead_time_h

#     preds_z500, preds_t850 = [], []

#     for fc_step in tqdm(range(max_fc_steps)):
#         # predict next state and update current with next
#         state = model.predict(state)
        
#         # Unnormalize
#         fc_z500 = state[:, :nlat*nlon].copy() * data_std.z.values + data_mean.z.values
#         fc_t850 = state[:, nlat*nlon:].copy() * data_std.t.values + data_mean.t.values
        
#         # Reshape
#         fc_z500 = fc_z500.reshape((-1, nlat, nlon))
#         fc_t850 = fc_t850.reshape((-1, nlat, nlon))
        
#         preds_z500.append(fc_z500)
#         preds_t850.append(fc_t850)

#     return [xr.DataArray(
#         np.array(fcs), 
#         dims=['lead_time', 'time', 'lat', 'lon'],
#         coords={
#             'lead_time': np.arange(lead_time_h, max_lead_time_h + lead_time_h, lead_time_h),
#             'time': z_test.time,
#             'lat': z_test.lat,
#             'lon': z_test.lon
#         }
#     ) for fcs in [preds_z500, preds_t850]]

In [121]:
# p, m = train_regression(LinearRegression, 
#                         6, 
#                         input_vars=['z', 't'], 
#                         output_vars=['z', 't'])

In [ ]:
# state = np.concatenate([data_test.z.values.reshape(-1, nlat*nlon), 
#                         data_test.t.values.reshape(-1, nlat*nlon)], 1)

In [ ]:
# fc_z500_6h_iter, fc_t850_6h_iter = create_iterative_lr(state, m)

In [ ]:
# fc_z500_6h_iter